In [25]:
from src.robinhood_api.api_access import CryptoAPITrading

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# Create an instance of the CryptoAPITrading which accesses the Robinhood API
api_trading_client = CryptoAPITrading()

api_trading_client.account.get_account()

{'account_number': '311061556262',
 'status': 'active',
 'buying_power': '1.4400',
 'buying_power_currency': 'USD'}

In [27]:
api_trading_client.account.get_holdings()

{'next': None,
 'previous': None,
 'results': [{'account_number': '311061556262',
   'asset_code': 'PEPE',
   'total_quantity': '115412.000000000000000000',
   'quantity_available_for_trading': '115412.000000000000000000'},
  {'account_number': '311061556262',
   'asset_code': 'AVAX',
   'total_quantity': '0.019000000000000000',
   'quantity_available_for_trading': '0.019000000000000000'}]}

In [28]:
# ask_inclusive_of_buy_spread is the market buy
# price is the price that robinhood shows

# want to make a chart that updates every 10 seconds wit the price

bid, ask = api_trading_client.market_data.get_estimated_price(symbol="BTC-USD", side="both", quantity=1.0)['results']

In [29]:
import os
import glob

def load_data(ticker, start_date=None, end_date=None, project_root='/Users/nathanielislas/PycharmProjects/CryptoTrading/'):
    # Get project root path
    #project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
    
    # Define data path relative to project root
    path_pattern = os.path.join(project_root, 'data', ticker, '*.parquet')
    files = glob.glob(path_pattern)

    df_list = []

    for file in files:
        date = os.path.basename(file).split('.')[0]

        # Load file if within date range or if no date range is specified
        if (start_date is None or end_date is None) or (start_date <= date <= end_date):
            df = pd.read_parquet(file)
            df_list.append(df)

    # Check if any data was loaded
    if not df_list:
        print(f"No data found for {ticker} in the specified date range.")
        return pd.DataFrame()  # Return empty DataFrame if no data

    # Concatenate and return combined DataFrame
    return pd.concat(df_list).reset_index(drop=True)

In [35]:
data = load_data(ticker='BTC-USD')
data

,symbol,price_bid,quantity,side_bid,bid_inclusive_of_sell_spread,sell_spread,timestamp,price_ask,side_ask,ask_inclusive_of_buy_spread,buy_spread
0,BTC-USD,86338.743745,1,bid,86333.33419,0.00006266,2025-03-07 23:26:49.591823235+00:00,86338.743745,ask,86344.1533,0.00006266
1,BTC-USD,86337.963105,1,bid,86332.18331,0.00006694,2025-03-07 23:26:50.719380159+00:00,86337.963105,ask,86343.7429,0.00006694
2,BTC-USD,86327.7823,1,bid,86322.8217,0.00005746,2025-03-07 23:26:51.843119525+00:00,86327.7823,ask,86332.7429,0.00005746
3,BTC-USD,86326.630695,1,bid,86318.05153,0.00009938,2025-03-07 23:26:52.973536748+00:00,86326.630695,ask,86335.20986,0.00009938
4,BTC-USD,86326.983635,1,bid,86320.68587,0.00007295,2025-03-07 23:26:54.097183180+00:00,86326.983635,ask,86333.2814,0.00007295
...,...,...,...,...,...,...,...,...,...,...,...
3619,BTC-USD,86104.81161,1,bid,86097.62322,0.00008348,2025-03-08 02:11:00.263753184+00:00,86104.81161,ask,86112,0.00008348
3620,BTC-USD,86109.36423,1,bid,86106.72846,0.00003061,2025-03-08 02:11:01.409797448+00:00,86109.36423,ask,86112,0.00003061
3621,BTC-USD,86111.7535285,1,bid,86102.554457,0.00010683,2025-03-08 02:11:02.839372514+00:00,86111.7535285,ask,86120.9526,0.00010683
3622,BTC-USD,86116.9402745,1,bid,86112.672449,0.00004956,2025-03-08 02:11:03.970464434+00:00,86116.9402745,ask,86121.2081,0.00004956
